In [ ]:
# Set local or cluster configuration
import os

if os.getenv("RSPY_LOCAL_MODE") == "1":
    RS_SERVER_ROOT_URL = "http://rs-server-catalog:8000"
    HEADERS={}
    local_mode = True
else:
    RS_SERVER_ROOT_URL = os.environ["RSPY_WEBSITE"]
    HEADERS={"headers": {"x-api-key": os.environ["RSPY_APIKEY"]}}
    local_mode = False

print(f"Using: {RS_SERVER_ROOT_URL}")

In [ ]:
RS_SERVER_ROOT_URL="http://localhost:8003"

In [ ]:
import json
import requests
import urllib.request

# owner:collection
owner="test"
collection=f"{owner}:sentinel1-grd"

# Clean old items and collections, if any
requests.delete(f"{RS_SERVER_ROOT_URL}/catalog/collections/{collection}", **HEADERS)

# Create a temp directory
import tempfile
tmp_dir = tempfile.TemporaryDirectory()
tmp_dirname = tmp_dir.name

In [ ]:
# Download a sample collection
urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/stactools-packages/sentinel1/main/examples/grd/collection.json", 
    f"{tmp_dirname}/collection.json")

# Open it, add the owner information
with open(f"{tmp_dirname}/collection.json", "r") as file:
    contents = json.load(file)
contents["owner"] = owner

# Add collection to catalog
response = requests.post(f"{RS_SERVER_ROOT_URL}/catalog/collections", json=contents, **HEADERS)
response.raise_for_status()
json.loads(response.content)

In [ ]:
# Add items into collection

urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/stactools-packages/sentinel1/main/examples/grd/S1A_EW_GRDM_1SDH_20221130T014342_20221130T014446_046117_058549/S1A_EW_GRDM_1SDH_20221130T014342_20221130T014446_046117_058549.json", 
    f"{tmp_dirname}/item1.json")
urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/stactools-packages/sentinel1/main/examples/grd/S1A_IW_GRDH_1SDV_20210809T173953_20210809T174018_039156_049F13/S1A_IW_GRDH_1SDV_20210809T173953_20210809T174018_039156_049F13.json", 
    f"{tmp_dirname}/item2.json")

with open(f"{tmp_dirname}/item1.json", "r") as file:
    item1_contents = json.load(file)
with open(f"{tmp_dirname}/item2.json", "r") as file:
    item2_contents = json.load(file)

print(json.dumps(item1_contents["assets"], indent=3))

response = requests.post(f"{RS_SERVER_ROOT_URL}/catalog/collections/{collection}/items", json=item1_contents, **HEADERS)
response.raise_for_status()
json.loads(response.content)

response = requests.post(f"{RS_SERVER_ROOT_URL}/catalog/collections/{collection}/items", json=item2_contents, **HEADERS)
response.raise_for_status()
json.loads(response.content)

In [ ]:
# Clean temp directory
tmp_dir.cleanup()

---
**NOTE**

STAC Swagger UI: http://localhost:8003/api.html

STAC browser: http://localhost:8081 (then load the STAC Catalog or API: `http://127.0.0.1:8003/catalog/sentinel1-grd`)

---